Sources: https://towardsdatascience.com/the-definitive-procedure-for-aligning-two-sets-of-3d-points-with-the-kabsch-algorithm-a7ec2126c87e 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.align_vectors.html 

# Given two Nx3 matrices, Rotate to minimize RMSD. Apply for computing RMSD for two wlcsim structures. 

In [1]:
import numpy as np
# import scipy
# from scipy import spatial
from scipy.spatial.transform import Rotation as R

In [70]:
def avg_col_subtract(a):
    """
    Subtract from each element the average of the whole column to center 
    the matrix at the origin
    
    Parameters
    ----------
    
    a: Nx3 np.array or np.matrix
        Original matrix
    
    Returns
    -------
    b: Nx3 np.array or np.matrix
        Mean subtracted matrix 
        
    Cites
    -----
    See bullet points 1 and 2: https://towardsdatascience.com/the-definitive-procedure-for-aligning-two-sets-of-3d-points-with-the-kabsch-algorithm-a7ec2126c87e 
    """
    b = a - a.mean(axis=0)
    return b

In [34]:
array_a = np.matrix([[1,2,3],[4,5,6],[6,23,6],[2,55,6],[41,35,6],[45,5,16]])
array_b = array_a + 13
rotate90 = np.matrix([[0, -1, 0],
                   [1, 0, 0],
                   [0, 0, 1]])
array_c = np.matmul(array_a, rotate90)
array_d = np.matmul(array_c, rotate90)

a_centered = avg_col_subtract(array_a)
b_centered = avg_col_subtract(array_b)
c_centered = avg_col_subtract(array_c)
d_centered = avg_col_subtract(array_d)

##### rssd is the square root of the weighted sum of the squared distances between the given sets of vectors after alignment.  It is equal to sqrt(2 * minimum_loss), where minimum_loss is the loss function evaluated for the found optimal rotation.

The distance between a vector and itself should be 0

In [35]:
estimated_rotation, rssd = R.align_vectors(a=array_a, b=array_a)
rssd==0

True

The distance between a vector and a translated version of itself should be 0

In [36]:
estimated_rotation, rssd = R.align_vectors(a=array_a, b=array_b)
rssd

53.590917643656574

The above failed because the matrices were not centered at the origin. Corrected below:

In [37]:
estimated_rotation, rssd = R.align_vectors(a=a_centered, b=b_centered)
rssd

1.3486991523486091e-06

The distance between a vector and a rotated version of itself should be 0

In [38]:
estimated_rotation, rssd = R.align_vectors(a=a_centered, b=c_centered)
rssd

0.0

In [39]:
estimated_rotation, rssd = R.align_vectors(a=array_a, b=array_d)
rssd

0.0

#### Check that differences can be detected:

Move one point along z axis. Verify that the rssd monotonically increases as the translation along the z increases.

In [53]:
for delta in np.arange(0,10):
    array_e = array_a + 13 + np.matrix([[0, 0, delta],[0, 0, 0],[0, 0, 0],[0, 0, 0],[0, 0, 0],[0, 0, 0]])
    e_centered = avg_col_subtract(array_e)
    estimated_rotation, rssd = R.align_vectors(a=a_centered, b=e_centered)
    print(rssd)

1.3486991523486091e-06
0.7443273964019765
1.4879472943013732
2.230892732984932
2.9731975852677732
3.7148965197470205
4.456024960472314
5.196619044134032
5.9367155748766764
6.6763519768948765


Combine as single formula in which 2 np.arrays can be given as input ans the rssd is returned.

In [59]:
#https://www.physicsforums.com/threads/rms-vs-rss-for-uncertainty.982466/ 
def rss_to_rmsd(rss,num_points):
    """
    Get RMSD from Root Sum of Squares by dividing by 1/n 
    where n is the number of points
    
    Parameters
    ----------
    rss: float
        root sum of squares computed from kabsch algorithm
        
    num_points: int
        Number of point in the original dataset 
        
    Returns
    -------
    rmsd: float
        root mean square deviation (aka rmse root mean square error)
    
    Cite (maybe verify with better cite)
    ----
    https://www.physicsforums.com/threads/rms-vs-rss-for-uncertainty.982466/ 
    """
    rmsd = rss / np.sqrt(num_points)
    return rmsd

In [68]:
def kabsh_algo_rmsd(positions_a, positions_b):
    """
    Given two sets of 3D positioning data, compute the RMSD after 
    optimally aligning using the kabsch algorithm 
    
    Parameters
    ---------
    positions_a: Nx3 np.array or np.matrix
        First set of positions 
    positions_b: Nx3 np.array or np.matrix
        Second set of positions 
        
    Returns
    -------
    rmsd : float 
        root-mean-square deviation 
    """
    number_points = np.shape(positions_a)[0]
    points_a_centered = avg_col_subtract(positions_a)
    points_b_centered = avg_col_subtract(positions_b)
    estimated_rotation, rssd = R.align_vectors(a=points_a_centered, b=points_b_centered)
    rmsd = rss_to_rmsd(rssd, number_points)
    return rmsd

In [69]:
for delta in np.arange(0,10):
    array_e = array_a + 13 + np.matrix([[0, 0, delta],[0, 0, 0],[0, 0, 0],[0, 0, 0],[0, 0, 0],[0, 0, 0]])
    e_centered = avg_col_subtract(array_e)
    rmsd = kabsh_algo_rmsd(a_centered, e_centered)
    print(rmsd)

0.0
0.30387038712652503
0.6074519391988661
0.9107581444493539
1.213802831396854
1.5166001534367755
1.8191645723772087
2.1215108409596013
2.423653984413593
2.7256092811022543
